In [56]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
# from seleniumwire import webdriver as wiredriver
from selenium.common.exceptions import NoSuchElementException

import os
import time
from io import StringIO

import requests
import pandas as pd

In [63]:
driver = webdriver.Chrome()
# url='https://election.gov.np/np/page/voter-list-db'
url = 'https://voterlist.election.gov.np/bbvrs1/index_2.php'
driver.get(url)

driver.implicitly_wait(0.5)

In [4]:
state_data=driver.find_element(by=By.XPATH, value="//select[@id='state']")
district_data=driver.find_element(by=By.XPATH, value="//select[@id='district']")
vdc_mun_data=driver.find_element(by=By.XPATH, value="//select[@id='vdc_mun']")
ward_data=driver.find_element(by=By.XPATH, value="//select[@id='ward']")
reg_center_data=driver.find_element(by=By.XPATH, value="//select[@id='reg_centre']")

submit_button = driver.find_element(By.ID, 'btnSubmit')

In [10]:
from selenium.webdriver.support.select import Select


Select(state_data).select_by_index(5)
Select(district_data).select_by_index(4)
Select(vdc_mun_data).select_by_index(3)
Select(ward_data).select_by_index(9)
Select(reg_center_data).select_by_index(1)

submit_button.click()

In [22]:
entries_dropdown = driver.find_element(by=By.XPATH, value="//select[@name='tbl_data_length']")
Select(entries_dropdown).select_by_index(3)

In [12]:
# driver.page_source

In [13]:
from bs4 import BeautifulSoup

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')

In [14]:
table = soup.find('table', class_ = 'bbvrs_data dataTable')

pd.read_html(str(table))[0]

C:\Users\ASUS\AppData\Local\Temp\ipykernel_27784\3593723493.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(str(table))[0]


,सि.नं.,मतदाता नं,मतदाताको नाम,उमेर (वर्ष),लिङ्ग,पति/पत्नीको नाम,पिता/माताको नाम,विवरण
0,1,16837317,अईभान डाँगी,49,पुरुष,विमला,भिम बहादुर/ निर्मला,NaN
1,2,16319606,अक्षय शाह,27,पुरुष,-,जनार्दन / राजेश्वरी,NaN
2,3,23527302,अग बहादुर वली,39,पुरुष,-,मनसुर / पुतला,NaN
3,4,1039501,अग्नु चौधरी,47,पुरुष,हिरोनी,सुरविर / रत्‍नी,NaN
4,5,3154649,अजमल खान,48,पुरुष,प्रविन,फते बहादुर/ राकिरा,NaN
...,...,...,...,...,...,...,...,...
95,96,1037621,अमृता कुमारी भण्डारी,35,महिला,गोपाल,लोक बहादुर/ रेशमी,NaN
96,97,25123808,अमृता कुमारी यरी,36,महिला,-,शोभाराम / डम्वर कुमारी,NaN
97,98,18052895,अमृता कुमारी रेउले,33,महिला,लाल बहादुर,दल बहादुर/ छवि,NaN
98,99,1038538,अम्बिका कुमारी नेपाली,35,महिला,दामोदर,छवीराम / कल्पना,NaN


In [99]:


driver = webdriver.Edge()
# url='https://election.gov.np/np/page/voter-list-db'
url = 'https://voterlist.election.gov.np/bbvrs1/index_2.php'
driver.get(url)

driver.implicitly_wait(1)

df = pd.DataFrame()

def post_submit_actions():
    entries_dropdown = driver.find_element(By.XPATH, '//select[@name="tbl_data_length"]')
    Select(entries_dropdown).select_by_value("100")

    def scraper():
        global df  
        page_content = driver.page_source
        soup = BeautifulSoup(page_content, 'html.parser')
        table = soup.find('table', class_='bbvrs_data dataTable')
        data = pd.read_html(StringIO(str(table)))[0]       
        df = pd.concat([df, data], ignore_index=True)

    while True:
        scraper()

        try:
            next_page_link = driver.find_element(By.CLASS_NAME, "paginate_enabled_next")

            if next_page_link:
                driver.execute_script("arguments[0].click();", next_page_link)
                # next_page_link.click()
        except NoSuchElementException:
            print("No more pages available")
            break

# Loop through each combination 
for state_index in range(6, 7):  #(1,8+1)
    try:
        for dist_index in range(4, 12): #(1, 14+1)
            try:
                for mun_index in range(1, 15): #(1, 18+1)
                    try:
                        for ward_index in range(1, 12): #(1, 11+1)
                            try:
                                for reg_centre_index in range(1, 4): #(1, 3+1)
                                    try:
                                        state_dropdown = driver.find_element(By.XPATH, '//select[@id="state"]')
                                        dist_dropdown = driver.find_element(By.XPATH, '//select[@id="district"]')
                                        mun_dropdown = driver.find_element(By.XPATH, '//select[@id="vdc_mun"]')
                                        ward_dropdown = driver.find_element(By.XPATH, '//select[@id="ward"]')
                                        reg_centre_dropdown = driver.find_element(By.XPATH, '//select[@id="reg_centre"]')
                                        submit_button = driver.find_element(By.ID, "btnSubmit")

                                        time.sleep(0.1)
                                        Select(state_dropdown).select_by_index(state_index)
                                        time.sleep(0.1)
                                        Select(dist_dropdown).select_by_index(dist_index)
                                        time.sleep(0.1)
                                        Select(mun_dropdown).select_by_index(mun_index)
                                        time.sleep(0.1)
                                        Select(ward_dropdown).select_by_index(ward_index)
                                        time.sleep(0.1)
                                        Select(reg_centre_dropdown).select_by_index(reg_centre_index)
                                        submit_button.click()

                                        post_submit_actions()
                                        
                                        go_back = driver.find_element(By.CLASS_NAME, "a_back")
                                        go_back.click()
                                        
                                    except NoSuchElementException:
                                        print("Element not found. Skipping...")
                            except:
                                break
                    except:
                        break
            except NoSuchElementException:
                break
    except NoSuchElementException:
        break
# driver.quit()


No more pages available
Element not found. Skipping...
Element not found. Skipping...
No more pages available
Element not found. Skipping...
Element not found. Skipping...
No more pages available
Element not found. Skipping...
Element not found. Skipping...
No more pages available
Element not found. Skipping...
Element not found. Skipping...
No more pages available
Element not found. Skipping...
Element not found. Skipping...
No more pages available
Element not found. Skipping...
Element not found. Skipping...
No more pages available
Element not found. Skipping...
Element not found. Skipping...
Element not found. Skipping...
Element not found. Skipping...
Element not found. Skipping...
Element not found. Skipping...
Element not found. Skipping...
Element not found. Skipping...
Element not found. Skipping...
Element not found. Skipping...
Element not found. Skipping...
Element not found. Skipping...
Element not found. Skipping...
Element not found. Skipping...
No more pages available
El

In [100]:
df.to_csv('from_jumla.csv', index=False)